# Tennis Players Ranking

## Objective: 

Objective of this project is to rank all Tennis Players based on the matches they played in the year of 2018. Statistics of all matches are given including their scores in all the sets of all matches. This project comprises 4 approaches to rank Tennis players and I have tried to make these approaches more robust sequentially.

## Importing libraries and Data

In [59]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import datetime

In [60]:
df = pd.read_csv("C:\\Users\\archd\\OneDrive\\Desktop\\UTR\\FINALutrdatascienceinternship2019.csv")

In [61]:
df.head()

,resultid,resultdate,winner1id,loser1id,winnerset1,winnerset2,winnerset3,winnerset4,winnerset5,loserset1,loserset2,loserset3,loserset4,loserset5
0,10979105,1/7/2018,3867,54552,6,6,0,0,0,4,3,0,0,0
1,10979106,1/7/2018,3671,4348,6,6,0,0,0,3,4,0,0,0
2,10979108,1/7/2018,52886,4021,6,7,0,0,0,4,5,0,0,0
3,10979110,1/7/2018,54846,3855,4,0,0,0,0,2,0,0,0,0
4,10979111,1/7/2018,3534,3688,6,6,0,0,0,3,2,0,0,0


In [62]:
df.tail()

,resultid,resultdate,winner1id,loser1id,winnerset1,winnerset2,winnerset3,winnerset4,winnerset5,loserset1,loserset2,loserset3,loserset4,loserset5
9453,12599577,7/25/2018,52886,53840,6,6,0,0,0,3,3,0,0,0
9454,12599578,7/25/2018,3481,28096,4,7,6,0,0,6,6,4,0,0
9455,12599579,7/25/2018,3853,218159,6,4,7,0,0,4,6,6,0,0
9456,12775191,8/12/2018,70522,405461,6,6,6,0,0,4,3,1,0,0
9457,12974164,9/3/2018,76935,69327,7,6,0,0,0,6,3,0,0,0


In [63]:
df.shape

(9458, 14)

The data contains statistics of total 9485 matches and I will rank the players based on these matches.

In [64]:
df.resultdate = pd.to_datetime(df.resultdate)

In [65]:
latest_match_day = max(df.resultdate.dt.dayofyear)
latest_match_day

246

In [66]:
df["Day_difference"] = latest_match_day - df.resultdate.dt.dayofyear

In [67]:
df.head()

,resultid,resultdate,winner1id,loser1id,winnerset1,winnerset2,winnerset3,winnerset4,winnerset5,loserset1,loserset2,loserset3,loserset4,loserset5,Day_difference
0,10979105,2018-01-07,3867,54552,6,6,0,0,0,4,3,0,0,0,239
1,10979106,2018-01-07,3671,4348,6,6,0,0,0,3,4,0,0,0,239
2,10979108,2018-01-07,52886,4021,6,7,0,0,0,4,5,0,0,0,239
3,10979110,2018-01-07,54846,3855,4,0,0,0,0,2,0,0,0,0,239
4,10979111,2018-01-07,3534,3688,6,6,0,0,0,3,2,0,0,0,239


## All players' player ids

In [68]:
all_players = np.concatenate((df.winner1id, df.loser1id))

In [69]:
all_players = np.unique(all_players)

In [70]:
len(all_players)

1052

I have total 1052 players and I will rank them accordingly.

## Number of Match Sets

In [71]:
df["sum_0"] = (df[["winnerset1", "winnerset2", "winnerset3", "winnerset4", "winnerset5", "loserset1", "loserset2", "loserset3", "loserset4", "loserset5"]] == 0).sum(axis=1)

In [72]:
df["sum_0"].head()

0    6
1    6
2    6
3    8
4    6
Name: sum_0, dtype: int64

In [73]:
def total_rounds(x):
    if (x == 10):
        return 0
    elif (x == 9 or x == 8):
        return 1
    elif (x == 7 or x == 6):
        return 2
    elif (x == 5 or x == 4):
        return 3
    elif (x == 3 or x == 2):
        return 4
    elif (x == 1 or x == 0):
        return 5
    else:
        return np.nan

In [74]:
df["total_rounds"] = df["sum_0"].apply(lambda x: total_rounds(x))

In [75]:
df["total_rounds"].head()

0    2
1    2
2    2
3    1
4    2
Name: total_rounds, dtype: int64

In [76]:
df["total_rounds"].value_counts()

2    5863
3    3385
4     125
5      73
1      12
Name: total_rounds, dtype: int64

In [77]:
df["total_rounds"].isnull().sum()

0

Let's find the score difference in all rounds

In [78]:
score_diff = pd.DataFrame()
for i in range(4,9):
    
    df_0 = df.iloc[:,i] - df.iloc[:, i+5]
    df_0 = df_0.reset_index(drop=True)
    score_diff = score_diff.reset_index(drop=True)
    score_diff = pd.concat([score_diff, df_0], axis = 1)

In [79]:
score_diff.columns = ["R1_score_diff", "R2_score_diff", "R3_score_diff", "R4_score_diff", "R5_score_diff"]

In [80]:
score_diff.head()

,R1_score_diff,R2_score_diff,R3_score_diff,R4_score_diff,R5_score_diff
0,2,3,0,0,0
1,3,2,0,0,0
2,2,2,0,0,0
3,2,0,0,0,0
4,3,4,0,0,0


Let's find how many sets winner won and how many loser won in a particular match based on score difference

In [81]:
df["winner_win_sets"] = (score_diff[score_diff.columns] > 0).sum(axis = 1)
df["loser_win_sets"] = (score_diff[score_diff.columns] < 0).sum(axis = 1)

In [82]:
df.head(10)

,resultid,resultdate,winner1id,loser1id,winnerset1,winnerset2,winnerset3,winnerset4,winnerset5,loserset1,loserset2,loserset3,loserset4,loserset5,Day_difference,sum_0,total_rounds,winner_win_sets,loser_win_sets
0,10979105,2018-01-07,3867,54552,6,6,0,0,0,4,3,0,0,0,239,6,2,2,0
1,10979106,2018-01-07,3671,4348,6,6,0,0,0,3,4,0,0,0,239,6,2,2,0
2,10979108,2018-01-07,52886,4021,6,7,0,0,0,4,5,0,0,0,239,6,2,2,0
3,10979110,2018-01-07,54846,3855,4,0,0,0,0,2,0,0,0,0,239,8,1,1,0
4,10979111,2018-01-07,3534,3688,6,6,0,0,0,3,2,0,0,0,239,6,2,2,0
5,10979112,2018-01-07,4047,51330,7,7,0,0,0,6,6,0,0,0,239,6,2,2,0
6,10979113,2018-01-07,3671,3867,7,6,7,0,0,6,7,6,0,0,239,4,3,2,1
7,10979114,2018-01-07,52886,53149,7,7,0,0,0,5,6,0,0,0,239,6,2,2,0
8,10979115,2018-01-07,94283,54846,7,7,0,0,0,6,6,0,0,0,239,6,2,2,0
9,10979116,2018-01-07,4047,3534,6,7,0,0,0,2,5,0,0,0,239,6,2,2,0


## Approach 1

Approach 1:
    To rank players, I have found out a score matrix which contains a score based on player's performance in each match. I have computed score using following method.
- The length of a match matters. If a match has 5 rounds, then it will reward or penalize a player more compared to a match having 3 rounds if a player wins or loses respectively.
- All the matches played by a player are considered and the recent the match, more weightage is given to a match.

Cons:
- If a player plays lots of matches, he can have high score.
- If a player wins or loses a most recent match, he will get rewarded (lucky guy!) or panelize (poor guy!) the highest.
- Only Match's verdict is considered. The number of sets won by a winner or looser are not considered and it does not matter if a looser loses by 3-2 or 4-1.

In [83]:
def approach1_score(player_df, player_id):
    player_df = player_df.reset_index(drop = True)
    matches = len(player_df)
    max_weight = matches / 100
    
    score = 0
    
    for i in range(len(player_df)):
        if player_df.winner1id[i] == player_id:
            score = score + (player_df.total_rounds[i] * max_weight * i / matches)
        if player_df.loser1id[i] == player_id:
            score = score - (player_df.total_rounds[i] * max_weight * i / matches)
    
    return score

In [84]:
approach1_df_score = pd.DataFrame(columns = ["Player_id", "score", "matches_played"])

In [85]:
approach1_df_score

,Player_id,score,matches_played


In [86]:
for i in all_players:
    temp_df = df[(df.winner1id == i) | (df.loser1id == i)]
    approach1_df_score = approach1_df_score.append({
                                                    "Player_id" : i,
                                                    "score" : approach1_score(temp_df, i),
                                                    "matches_played" : len(temp_df)
                                                    }, ignore_index=True)

In [87]:
approach1_df_score.head(10)

,Player_id,score,matches_played
0,3456.0,7.25,32.0
1,3458.0,4.84,40.0
2,3460.0,0.90,33.0
3,3462.0,2.73,29.0
4,3464.0,2.63,40.0
5,3465.0,-1.64,33.0
6,3466.0,2.31,29.0
7,3468.0,3.38,37.0
8,3469.0,0.61,31.0
9,3474.0,-2.33,20.0


In [88]:
approach1_df_score = approach1_df_score.sort_values(by = ["score"], ascending= False)
approach1_df_score["rank"] = ss.rankdata(approach1_df_score["score"], method='dense')
approach1_df_score["rank"] = max(approach1_df_score["rank"]) + 1 - approach1_df_score["rank"]

In [89]:
approach1_df_score.head(10)

,Player_id,score,matches_played,rank
170,3927.0,14.19,49.0,1
291,4584.0,13.95,47.0,2
501,52294.0,13.70,43.0,3
236,4205.0,13.59,35.0,4
280,4515.0,13.47,42.0,5
105,3745.0,10.99,46.0,6
879,206792.0,10.86,52.0,7
12,3478.0,10.73,34.0,8
334,5059.0,10.71,36.0,9
85,3695.0,10.64,38.0,10


In [90]:
approach1_df_score.to_excel("C:\\Users\\archd\\OneDrive\\Desktop\\UTR\\approach1_df_score.xlsx")

## Approach 2

Approach 2:
    To rank players, I have found out a score matrix which contains a score based on player's performance in each match. I have computed score using following method.
- The length of a match matters. If a match has 5 rounds, then it will reward or penalize a player more compared to a match having 3 rounds if a player wins or loses respectively.
- Only recent 30 matches played by a player are considered and the recent the match, more weightage is given to a match.

Cons:
- If a player wins or loses a most recent match, he will get rewarded (lucky guy!) or panelize (poor guy!) the highest.
- Only Match's verdict is considered. The number of sets won by a winner or looser are not considered and it does not matter if a looser loses by 3-2 or 4-1.

In [91]:
def approach2_score(player_df, player_id):
    matches = len(player_df)
    if (matches>30):
        player_df = player_df.iloc[matches-30 : matches, :]
    
    player_df = player_df.reset_index(drop = True)
    max_weight = 30 / 100
    
    score = 0
    
    for i in range(len(player_df)):
        if player_df.winner1id[i] == player_id:
            score = score + (player_df.total_rounds[i] * max_weight * i / 30)
        if player_df.loser1id[i] == player_id:
            score = score - (player_df.total_rounds[i] * max_weight * i / 30)
    
    return score

In [92]:
approach2_df_score = pd.DataFrame(columns = ["Player_id", "score"])

In [93]:
approach2_df_score

,Player_id,score


In [94]:
for i in all_players:
    temp_df = df[(df.winner1id == i) | (df.loser1id == i)]
    approach2_df_score = approach2_df_score.append({
                                                    "Player_id" : i,
                                                    "score" : approach2_score(temp_df, i)
                                                    }, ignore_index=True)

In [95]:
approach2_df_score.head(10)

,Player_id,score
0,3456.0,6.16
1,3458.0,2.64
2,3460.0,1.11
3,3462.0,2.73
4,3464.0,1.63
5,3465.0,-1.38
6,3466.0,2.31
7,3468.0,2.11
8,3469.0,0.45
9,3474.0,-2.33


In [96]:
approach2_df_score = approach2_df_score.sort_values(by = ["score"], ascending= False)
approach2_df_score["rank"] = ss.rankdata(approach2_df_score["score"], method='dense')
approach2_df_score["rank"] = max(approach2_df_score["rank"]) + 1 - approach2_df_score["rank"]

In [97]:
approach2_df_score.head(10)

,Player_id,score,rank
236,4205.0,10.00,1
12,3478.0,9.20,2
334,5059.0,7.34,3
280,4515.0,7.09,4
85,3695.0,6.78,5
501,52294.0,6.74,6
0,3456.0,6.16,7
170,3927.0,6.00,8
84,3691.0,5.52,9
165,3907.0,5.42,10


In [98]:
approach2_df_score.to_excel("C:\\Users\\archd\\OneDrive\\Desktop\\UTR\\approach2_df_score.xlsx")

## Approach 3

Approach 3:
    To rank players, I have found out a score matrix which contains a score based on player's performance in each match. I have computed score using following method.
- The length of a match matters. If a match has 5 rounds, then it will reward or penalize a player more compared to a match having 3 rounds if a player wins or loses respectively.
- Only recent 30 matches played by a player are considered. Recent match get high weightage compared to past matches but weightage is a step function with step length is equal to 3. So, a player's rank does not fall or climb rapidly just because good or poor performance in the last match. 

Cons:
- Only Match's verdict is considered. The number of sets won by a winner or looser are not considered and it does not matter if a looser loses by 3-2 or 4-1.

In [99]:
def approach3_score(player_df, player_id):
    matches = len(player_df)
    if (matches>30):
        player_df = player_df.iloc[matches-30 : matches, :]
    
    player_df = player_df.reset_index(drop = True)
    max_weight = 30 / 100
    
    score = 0
    
    for i in range(len(player_df)):
        if player_df.winner1id[i] == player_id:
            score = score + (player_df.total_rounds[i] * max_weight * (int(i/3) + 1) / 10)
        if player_df.loser1id[i] == player_id:
            score = score - (player_df.total_rounds[i] * max_weight * (int(i/3) + 1) / 10)
    
    return score

In [100]:
approach3_df_score = pd.DataFrame(columns = ["Player_id", "score"])

In [101]:
approach3_df_score

,Player_id,score


In [102]:
for i in all_players:
    temp_df = df[(df.winner1id == i) | (df.loser1id == i)]
    approach3_df_score = approach3_df_score.append({
                                                    "Player_id" : i,
                                                    "score" : approach3_score(temp_df, i)
                                                    }, ignore_index=True)

In [103]:
approach3_df_score.head(10)

,Player_id,score
0,3456.0,7.38
1,3458.0,2.91
2,3460.0,1.23
3,3462.0,3.03
4,3464.0,1.80
5,3465.0,-1.59
6,3466.0,2.61
7,3468.0,2.31
8,3469.0,0.81
9,3474.0,-2.58


In [104]:
approach3_df_score = approach3_df_score.sort_values(by = ["score"], ascending= False)
approach3_df_score["rank"] = ss.rankdata(approach3_df_score["score"], method='dense')
approach3_df_score["rank"] = max(approach3_df_score["rank"]) + 1 - approach3_df_score["rank"]

In [105]:
approach3_df_score.head(10)

,Player_id,score,rank
236,4205.0,11.43,1
12,3478.0,9.93,2
334,5059.0,8.46,3
501,52294.0,7.89,4
280,4515.0,7.80,5
85,3695.0,7.65,6
0,3456.0,7.38,7
170,3927.0,6.54,8
84,3691.0,6.36,9
165,3907.0,6.06,10


In [106]:
approach3_df_score.to_excel("C:\\Users\\archd\\OneDrive\\Desktop\\UTR\\approach3_df_score.xlsx")

## Approach 4

Approach 4:
    To rank players, I have found out a score matrix which contains a score based on player's performance in each match. I have computed score using following method.
- The length of a match matters. If a match has 5 rounds, then it will reward or penalize a player more compared to a match having 3 rounds if a player wins or loses respectively.
- Only recent 30 matches played by a player are considered. Recent match get high weightage compared to past matches but weightage is a step function with step length is equal to 3. So, a player's rank does not fall or climb rapidly just because good or poor performance in the last match. 
- The number of sets won by a winner or looser are considered so if a player loses by 3-2, he gets panelize less compared to if he loses by 4-1. Same way, winner gets rewarded more if he wins a match by 4-1 compared to 3-2. 

In [107]:
def approach4_score(player_df, player_id):
    matches = len(player_df)
    if (matches>30):
        player_df = player_df.iloc[matches-30 : matches, :]
    
    player_df = player_df.reset_index(drop = True)
    max_weight = 30 / 100
    
    score = 0
    
    for i in range(len(player_df)):
        if player_df.winner1id[i] == player_id:
            score = score + (player_df.winner_win_sets[i] * max_weight * (int(i/3) + 1) / 30)
        if player_df.loser1id[i] == player_id:
            score = score - (player_df.winner_win_sets[i] * max_weight * (int(i/3) + 1) / 30)
    
    return score

In [108]:
approach4_df_score = pd.DataFrame(columns = ["Player_id", "score"])

In [109]:
approach4_df_score

,Player_id,score


In [110]:
for i in all_players:
    temp_df = df[(df.winner1id == i) | (df.loser1id == i)]
    approach4_df_score = approach4_df_score.append({
                                                    "Player_id" : i,
                                                    "score" : approach4_score(temp_df, i)
                                                    }, ignore_index=True)

In [111]:
approach4_df_score.head(10)

,Player_id,score
0,3456.0,2.51
1,3458.0,0.98
2,3460.0,0.34
3,3462.0,0.51
4,3464.0,0.59
5,3465.0,-0.35
6,3466.0,0.78
7,3468.0,0.67
8,3469.0,0.05
9,3474.0,-0.54


In [112]:
approach4_df_score = approach4_df_score.sort_values(by = ["score"], ascending= False)
approach4_df_score["rank"] = ss.rankdata(approach4_df_score["score"], method='dense')
approach4_df_score["rank"] = max(approach4_df_score["rank"]) + 1 - approach4_df_score["rank"]

In [113]:
approach4_df_score.head(10)

,Player_id,score,rank
236,4205.0,3.62,1
12,3478.0,3.03,2
334,5059.0,2.63,3
0,3456.0,2.51,4
501,52294.0,2.08,5
280,4515.0,2.03,6
85,3695.0,2.00,7
165,3907.0,1.98,8
381,28096.0,1.82,9
89,3700.0,1.81,10


In [114]:
approach4_df_score.to_excel("C:\\Users\\archd\\OneDrive\\Desktop\\UTR\\approach4_df_score.xlsx")

## Comparision

In [115]:
approach1_df_score = approach1_df_score.sort_index()
approach2_df_score = approach2_df_score.sort_index()
approach3_df_score = approach3_df_score.sort_index()
approach4_df_score = approach4_df_score.sort_index()

In [116]:
Ranks_compare_df = pd.DataFrame(columns = ["Player_id", "approach1_rank", "approach2_rank", "approach3_rank", "approach4_rank"])

In [117]:
Ranks_compare_df

,Player_id,approach1_rank,approach2_rank,approach3_rank,approach4_rank


In [118]:
Ranks_compare_df["Player_id"] = all_players
Ranks_compare_df["approach1_rank"] = approach1_df_score["rank"]
Ranks_compare_df["approach2_rank"] = approach2_df_score["rank"]
Ranks_compare_df["approach3_rank"] = approach3_df_score["rank"]
Ranks_compare_df["approach4_rank"] = approach4_df_score["rank"]

In [119]:
Ranks_compare_df.head(10)

,Player_id,approach1_rank,approach2_rank,approach3_rank,approach4_rank
0,3456,20,7,7,4
1,3458,46,71,65,49
2,3460,188,152,127,114
3,3462,100,69,60,94
4,3464,102,116,105,86
5,3465,609,589,428,322
6,3466,114,88,77,64
7,3468,77,98,86,75
8,3469,213,222,163,187
9,3474,653,657,476,356


In [120]:
Ranks_compare_df.to_excel("C:\\Users\\archd\\OneDrive\\Desktop\\UTR\\Ranks_compare_df.xlsx")